In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
DRIVE_PATH='/content/drive/MyDrive/CS_DS541/'

Mounted at /content/drive/


Cleaning Songs CSV into a a set of features

In [ ]:
import dask.dataframe as dd
import dask.array as da
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.ensemble import RandomForestClassifier

In [ ]:
feat = dd.read_parquet(DRIVE_PATH+'/data/feat_parquet.pq').astype('float32')
raw = dd.read_parquet(DRIVE_PATH+'/data/raw_parquet.pq')
audioFeatures = ['danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo']
features = [ 'duration_ms' ] + audioFeatures
featTF = [feature+'_tf' for feature in features]

try:
  embed = dd.read_parquet(DRIVE_PATH+'/data/embed.pq')[featTF].astype('float32')
except:
  # Queue Transform
  for feature in features:
    feat[feature+'_tf'] = (feat[feature] - feat[feature].mean())/feat[feature].std()
  feat.to_parquet(DRIVE_PATH+'/data/embed.pq')
  embed = feat.astype('float32')


embed['genre'] = raw.genre
gen_idx = embed.groupby('genre').mean().index
gen_map = dict(zip(gen_idx,np.arange(len(gen_idx))))
embed['gen_code'] = embed.genre.map(lambda gen: gen_map[gen],meta=(object,object)).persist()

In [ ]:
embed = embed.drop(labels=embed.columns[10], axis=1)

a, b, c = embed.random_split([0.8, 0.1, 0.1], random_state=904)

del embed, feat, raw, gen_idx, gen_map

In [ ]:
train_data = a[featTF].compute()
train_labels = a[a.columns[10]].compute()

val_data = b[featTF].compute()
val_labels = b[b.columns[10]].compute()


del a, b, c

In [ ]:
from sklearn import tree
clf2 = tree.DecisionTreeClassifier(max_depth = 16, criterion = 'entropy')
clf2 = clf2.fit(train_data, train_labels)
print(clf2.score(val_data, val_labels))